In [1]:
import pandas as pd
import io
import requests

YearJoined = pd.read_csv("https://raw.githubusercontent.com/PracticalTimeSeriesAnalysis/BookRepo/master/Ch02/data/year_joined.csv")
emails = pd.read_csv("https://raw.githubusercontent.com/PracticalTimeSeriesAnalysis/BookRepo/master/Ch02/data/emails.csv")
donations = pd.read_csv("https://raw.githubusercontent.com/PracticalTimeSeriesAnalysis/BookRepo/master/Ch02/data/donations.csv")

In [3]:
YearJoined.head()

,user,userStats,yearJoined
0,0,silver,2014
1,1,silver,2015
2,2,silver,2016
3,3,bronze,2018
4,4,silver,2018


In [4]:
emails.head()

,emailsOpened,user,week
0,3.0,1.0,2015-06-29 00:00:00
1,2.0,1.0,2015-07-13 00:00:00
2,2.0,1.0,2015-07-20 00:00:00
3,3.0,1.0,2015-07-27 00:00:00
4,1.0,1.0,2015-08-03 00:00:00


In [5]:
donations.head()

,amount,timestamp,user
0,25.0,2017-11-12 11:13:44,0.0
1,50.0,2015-08-25 19:01:45,0.0
2,25.0,2015-03-26 12:03:47,0.0
3,50.0,2016-07-06 12:24:55,0.0
4,50.0,2016-05-11 18:13:04,1.0


In [6]:
YearJoined.groupby('user').count().groupby('userStats').count()

,yearJoined
userStats,
1,1000


1000명의 모든 회원이 단 하나의 상태만 가진다는 것을 알 수 있음

In [9]:
emails[emails.emailsOpened<1]

,emailsOpened,user,week


널이 아예 발생하지 않았거나 모든 회원이 이메일을 열람하거나 둘 중 하나의 이벤트가 발생함

In [11]:
emails[emails.user == 998]

,emailsOpened,user,week
25464,1.0,998.0,2017-12-04 00:00:00
25465,3.0,998.0,2017-12-11 00:00:00
25466,3.0,998.0,2017-12-18 00:00:00
25467,3.0,998.0,2018-01-01 00:00:00
25468,3.0,998.0,2018-01-08 00:00:00
25469,2.0,998.0,2018-01-15 00:00:00
25470,3.0,998.0,2018-01-22 00:00:00
25471,2.0,998.0,2018-01-29 00:00:00
25472,3.0,998.0,2018-02-05 00:00:00
25473,3.0,998.0,2018-02-12 00:00:00


코드 결과에서 일부 주가 누락된 것을 확인

In [14]:
import datetime

date_time_str_max = max(emails[emails.user == 998].week)
date_time_str_min = min(emails[emails.user == 998].week)

date_time_obj_max = datetime.datetime.strptime(date_time_str_max,'%Y-%m-%d %H:%M:%S')
date_time_obj_min = datetime.datetime.strptime(date_time_str_min,'%Y-%m-%d %H:%M:%S')

(date_time_obj_max - date_time_obj_min).days/7

25.0

998번 user의 첫 이벤트 시점과 마지막 이벤트 시점 사이에는 25개의 주가 있음

-> 1을 더해서 모든 주는 총 26주여야함

In [16]:
emails[emails.user == 998].shape

(24, 3)

데이터상에는 24개의 주만 기록되어 있음

In [17]:
#set을 이용해 각 열의 유니크한 값 목록 생성

complete_idx = pd.MultiIndex.from_product((set(emails.week),set(emails.user)))

In [19]:
# 재색인 메서드 : reindx
# 색인 재설정 메서드 : reset_index
all_email = emails.set_index(['week','user']).reindex(complete_idx, fill_value=0).reset_index()

#재설정된 색인에 의해 생성된 열의 이름을 붙여줌
all_email.columns = ['week','user','emailsOpened']

In [20]:
all_email[all_email.user == 998].sort_values('week')

,week,user,emailsOpened
44197,2015-02-09 00:00:00,998.0,0.0
77076,2015-02-16 00:00:00,998.0,0.0
83005,2015-02-23 00:00:00,998.0,0.0
62523,2015-03-02 00:00:00,998.0,0.0
39885,2015-03-09 00:00:00,998.0,0.0
...,...,...,...
51743,2018-04-30 00:00:00,998.0,3.0
10779,2018-05-07 00:00:00,998.0,3.0
76537,2018-05-14 00:00:00,998.0,3.0
71686,2018-05-21 00:00:00,998.0,3.0


998번 user가 회원가입 전의 주들은 모두 0으로 채워져 있음

In [21]:
cutoff_dates = emails.groupby('user').week.agg(['min','max']).reset_index()
cutoff_dates = cutoff_dates.reset_index()

이메일 데이터프레임을 회원별로 그룹화하여 start_date 및 end_date를 결정하고 각 회원이 이메일을 수신해온 주의 시작과 끝을 알 수 있음

In [25]:
import warnings
warnings.filterwarnings('ignore')

for _, row in cutoff_dates.iterrows():
    user = row['user']
    start_date = row['min']
    end_date = row['max']
    all_email.drop(all_email[all_email.user == user][all_email.week < start_date].index, inplace=True)
    all_email.drop(all_email[all_email.user == user][all_email.week > end_date].index, inplace=True)

donation 데이터를 주 단위로 다운 샘플링 하여 기본 시계열로 바꾸면 이메일 데이터와 비교할 수 있음

In [28]:
donations.timestamp = pd.to_datetime(donations.timestamp)
donations.set_index('timestamp',inplace=True)
agg_don = donations.groupby('user').apply(lambda df: df.amount.resample('W-MON').sum().dropna())

In [29]:
agg_don

user   timestamp 
0.0    2015-03-30      25.0
       2015-04-06       0.0
       2015-04-13       0.0
       2015-04-20       0.0
       2015-04-27       0.0
                      ...  
995.0  2017-09-11       0.0
       2017-09-18       0.0
       2017-09-25       0.0
       2017-10-02    1000.0
998.0  2018-01-08      50.0
Name: amount, Length: 32352, dtype: float64

In [ ]:
merge_df = pd.DataFrame()

for user, user_email in all_email.groupby('user'):
    #특정 회원의 기분 데이터 추출
    user_donations = agg_don[agg_don.index.get_level_values('user') == user]
    
    user_donations = user_donations.droplevel(0)
    
    #기부데이터의 색인을 timestamp로 설정
    user_email = all_email[all_email.user == user]
    user_email.sort_values('week',inplace=True)
    user_email.set_index('week',inplace=True)
    
    